# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [38]:
print(salaries.shape)
salaries.head()

(33183, 8)


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [37]:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
h_s = salaries['Salary or Hourly'].value_counts()
h_s

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
dep = salaries.Department.value_counts()
dep

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
hourly = salaries.loc[salaries['Salary or Hourly'] == 'Hourly']


In [8]:
data = hourly['Hourly Rate']
data.dropna()

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [9]:
data = data.to_numpy()

In [10]:
result = stats.ttest_1samp(data, 30)
result

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

In [11]:
#Se rechaza la hipótesis nula si el pvalue es igual o menor que 0,05. 
#Como este caso el pvalue es mayor, aceptamos la hipótesis.

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [12]:
policies = salaries.loc[salaries['Department'] == 'POLICE']

In [13]:
police = policies['Annual Salary']
police.dropna()

0        101442.0
1         94122.0
9         93354.0
11        84054.0
12        87006.0
           ...   
33177     72510.0
33178     72510.0
33179     48078.0
33180     90024.0
33181     93354.0
Name: Annual Salary, Length: 13404, dtype: float64

In [14]:
police = police.notnull() 

In [15]:
police.isnull().value_counts()

False    13414
Name: Annual Salary, dtype: int64

In [16]:
police = police.to_numpy()

In [17]:
p_result = stats.ttest_1samp(police,86000)
p_result

Ttest_1sampResult(statistic=-364919826.5437878, pvalue=0.0)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [18]:
ct = pd.crosstab(index=salaries['Department'], columns=salaries['Salary or Hourly'])

In [19]:
ct.sort_values(by=['Hourly'],ascending=False).head()

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215


In [20]:
# The department is STREETS & SAN.

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [21]:
ss = salaries.loc[salaries['Department'] == 'STREETS & SAN']


In [22]:
s_s = ss['Hourly Rate']
s_s.dropna()

7        35.60
21       21.43
24       35.60
36       36.21
39       35.60
         ...  
33106    36.13
33107    35.60
33147    35.60
33149    36.21
33156    22.12
Name: Hourly Rate, Length: 1862, dtype: float64

In [23]:
s_s = s_s.notnull()

In [24]:
s_s = s_s.to_numpy()

In [25]:
ss_result = stats.ttest_1samp(s_s, 35)
ss_result

Ttest_1sampResult(statistic=-4448.467241449082, pvalue=0.0)

In [26]:
#Se rechaza la hipótesis nula si el pvalue es igual o menor que 0,05. 
#Como este caso el pvalue es cero, rechazamos la hipótesis.

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [27]:
data #this is the clean array of hourly rate for hourly workeys

array([19.86, 46.1 , 35.6 , ..., 35.6 , 46.35, 48.85])

In [28]:
np.mean(data)

32.78855771628024

In [29]:
t_interval =stats.t.pdf(x=data, df=data, loc=32.78855771628024)
t_interval

array([2.74079641e-11, 2.90158002e-17, 1.00988863e-02, ...,
       1.00988863e-02, 1.30813035e-17, 5.10580418e-21])

In [30]:
stats.sem(t_interval, axis=0, ddof=-1, nan_policy='propagate')

0.0006026010270636355

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [31]:
np.mean(police)

0.9992545102132101

In [32]:
t_all =stats.t.pdf(x=police, df=police, loc=0.9992545102132101)
t_all


array([0.31830971, 0.31830971, 0.31830971, ..., 0.31830971, 0.31830971,
       0.31830971])

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [33]:
c4 = pd.crosstab(index=salaries['Salary or Hourly'], columns=salaries['Salary or Hourly'])

In [34]:
c4

Salary or Hourly,Hourly,Salary
Salary or Hourly,,
Hourly,8022,0
Salary,0,25161


In [35]:
a = proportions_ztest(count=8022, nobs=33183, value=0.25)

In [36]:
a

(-3.5099964213703005, 0.0004481127249057967)